<a href="https://colab.research.google.com/github/njgeorge000158/deep-learning-challenge/blob/main/AlphabetSoupCharityOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [701]:
#*******************************************************************************************
 #
 #  File Name:  AlphabetSoupCharityOptimization.ipynb
 #
 #  File Description:
 #      This interactive Python notebook,AlphabetSoupCharityOptimization.ipynb, reads
 #      a csv file, LendingData.csv, in the Resources folder, and uses Python and neural
 #      networks methods to use the features in the provided dataset to create a binary
 #      classifier that can predict whether applicants for charity donations will be
 #      successful if funded by the nonprofit foundation, Alphabet Soup.
 #
 #      I reinstalled the scikit-learn module using the following commands to allow the
 #      RandomOverSampler function to work:
 #
 #      pip3 uninstall scikit-learn
 #      pip3 install scikit-learn==1.2.2
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/03/2023      Initial Development                     N. James George
 #
 #******************************************************************************************/

from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.insert(0,'/content/drive/My Drive/deep-learning-challenge')

!pip install -q dataframe_image
!pip install aspose-words
!pip install hvplot
!pip install silence-tensorflow
!pip install keras-tuner

import PyConstants as constant
import PyFunctions as function
import PyLogConstants as log_constant
import PyLogFunctions as log_function
import PyLogSubRoutines as log_subroutine
import PySubRoutines as subroutine

import AlphabetSoupCharityOptimizationSearchFunctions as local_function

import pandas as pd

import json

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [702]:
CONSTANT_LOCAL_FILE_NAME \
    = 'AlphabetSoupCharityOptimization.ipynb'

CONSTANT_CHARITY_DATA_CSV_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/CharityData.csv'

CONSTANT_OPTIMIZATION_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/AlphabetSoupCharityOptimization.keras'

log_subroutine \
    .SetLogMode \
        (False)

log_subroutine \
    .SetDebugMode \
        (False)

log_subroutine \
    .SetImageMode \
        (False)

log_subroutine \
    .BeginProgramExecution \
        ('AlphabetSoupCharityOptimization')

# <br> **Section 1: Data Acquistion**

## **1.1: Import and Read Charity Data**

In [703]:
charityDataFrame \
    = function. \
        ReturnCSVFileAsDataFrame \
            (CONSTANT_CHARITY_DATA_CSV_FILE_PATH)


log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

## **1.2: Display Charity Data**

In [704]:
captionString \
    = 'Table 1.2.1: Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
"10,520,599",BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
"10,531,628",AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
"10,547,893",ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
"10,553,066",SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
"10,556,103",GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
"10,556,855",MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
"10,558,440",FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
"10,566,033",ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
"10,570,430",ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
"10,571,689",INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Drop the Non-Beneficial ID Columns**

In [705]:
charityDataFrame \
    .drop \
        (['EIN'],
         axis = 1,
         inplace = True)

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

In [706]:
captionString \
    = 'Table 1.2.2: Modified Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


## **2.2: Determine the Number of Unique Values in Each Column**

In [707]:
numberOfUniqueValuesInEachColumnSeries \
    = charityDataFrame \
        .nunique()

log_function \
    .DebugReturnObjectWriteObject \
        (numberOfUniqueValuesInEachColumnSeries)

In [708]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(numberOfUniqueValuesInEachColumnSeries))

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


## **2.3: NAME Binning**

### **Look at NAME Value Counts for Binning**

In [709]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

In [710]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(nameValueCountsForBinningSeries))

PARENT BOOSTER USA INC                                              1260
TOPS CLUB INC                                                        765
UNITED STATES BOWLING CONGRESS INC                                   700
WASHINGTON STATE UNIVERSITY                                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                      408
                                                                    ... 
POM-POMS CASTLE                                                        1
FOUNDATION FOR INTERVENTIONAL RADIOLOGICAL RESEARCH SUPP & TEACH       1
DARE U TO CARE OUTREACH MINISTRY                                       1
GEFFEN PLAYHOUSE INC                                                   1
WATERHOUSE CHARITABLE TR                                               1
Name: NAME, Length: 19568, dtype: int64


### **Choose a Cutoff Value and Create a List of Names to be Replaced**

In [711]:
names_to_replace \
    = list \
        (nameValueCountsForBinningSeries \
             [nameValueCountsForBinningSeries < 5].index)

log_function \
    .DebugReturnObjectWriteObject \
        (names_to_replace)

In [712]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(names_to_replace))

['NATIONAL SOCIETY OF THE CHILDREN OF THE AMERICAN REVOLUTION', 'FREE & ACCEPTED MASONS OF LOUISIANA GRAND LODGE', 'TENNESSEE MUSIC EDUCATION ASSOCIATION', 'COMMEMORATIVE AIR FORCE', 'JEFFERSON CITY SENIOR HIGH SCHOOL PARENT TEACHERS ORGANIZATION', 'TXCPSO INC', 'EPSILON PI TAU INC', 'NATIONAL SORORITY OF PHI DELTA KAPPA INC', 'VETERANS OF FOREIGN WARS OF THE UNITED STATES DEPT OF KANSAS', 'PHI DELTA KAPP INTERNATIONAL INC', 'AMERICAN SOCIETY OF HEATING REFRIGERATING & A-C ENGINEERS', 'ORDER OF THE EASTERN STAR OF MISSOURI GRAND CHAPTER', 'LIFE STEPS FOUNDATION INC', 'OMEGA PSI PHI FRATERNITY', 'LOUISIANA 4-H FOUNDTION', 'COMPUTER USING EDUCATORS', 'ANCIENT FREE AND ACCEPTED MASONS OF OKLAHOMA', 'AMERICAN COACH ASSOCIATION', 'AMATEUR ATHLETIC UNIONS OF THE US', 'STANDUP PARENTING INC', 'TEXAS STATE BRANCH 948 NATIONAL ASSOCIATION OF POSTAL SUPERVISORS', 'SUNRISE 2000 INC', 'JUNIOR ROLLER DERBY ASSOCIATION', 'SIGMA GAMMA RHO SORORITY INC', 'EXPERIMENTAL AIRCRAFT ASSOCIATION', 'INSTITUTE

### **Replace Application Types in DataFrame**

In [713]:
for name in names_to_replace:

    charityDataFrame['NAME'] \
        = charityDataFrame['NAME'] \
            .replace \
                (name, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [714]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

In [715]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(nameValueCountsForBinningSeries))

Other                                                            19803
PARENT BOOSTER USA INC                                            1260
TOPS CLUB INC                                                      765
UNITED STATES BOWLING CONGRESS INC                                 700
WASHINGTON STATE UNIVERSITY                                        492
                                                                 ...  
VETERANS OF FOREIGN WARS OF THE US DEPT OF TEXAS AUXILIARY           5
COUNTRY DANCE AND SONG SOCIETY INC                                   5
INTERNATIONAL ORDER OF THE RAINBOW GIRLS THE SUPREME ASSEMBLY        5
NATIONAL ASSOCIATION OF COMPOSERS USA                                5
PTA HAWAII CONGRESS                                                  5
Name: NAME, Length: 403, dtype: int64


## **2.3: APPLICATION_TYPE Binning**

### **Look at APPLICATION_TYPE Value Counts for Binning**

In [716]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [717]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


### **Choose a Cutoff Value and Create a List of Application Types to be Replaced**

In [718]:
application_types_to_replace \
    = list \
        (applicationTypeValueCountsForBinningSeries \
             [applicationTypeValueCountsForBinningSeries < 500].index)

log_function \
    .DebugReturnObjectWriteObject \
        (application_types_to_replace)

In [719]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(application_types_to_replace))

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### **Replace Application Types in DataFrame**

In [720]:
for applicationTypes in application_types_to_replace:

    charityDataFrame['APPLICATION_TYPE'] \
        = charityDataFrame['APPLICATION_TYPE'] \
            .replace \
                (applicationTypes, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [721]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [722]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


## **2.4: CLASSIFICATION Binning**

### **Look at CLASSIFICATION Value Counts for Binning**

In [723]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [724]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


### **Look at CLASSIFICATION Value Counts > 1**

In [725]:
classificationValueCountsForBinningSeries \
    [classificationValueCountsForBinningSeries > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

### **Choose a Cutoff Value and Create a List of Classifications to be Replaced**

In [726]:
classifications_to_replace \
    = list \
        (classificationValueCountsForBinningSeries \
             [classificationValueCountsForBinningSeries < 100].index)

log_function \
    .DebugReturnObjectWriteObject \
        (classifications_to_replace)

### **Replace Classifications in DataFrame**

In [727]:
for classification in classifications_to_replace:

    charityDataFrame['CLASSIFICATION'] \
        = charityDataFrame['CLASSIFICATION'] \
            .replace \
                (classification, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [728]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [729]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


## **2.5: Final Preprocessing**

### **Convert Categorical Data to Numeric with `pd.get_dummies`**

In [730]:
dummiesDataFrame \
    = pd.get_dummies \
        (charityDataFrame)

log_function \
    .DebugReturnObjectWriteObject \
        (dummiesDataFrame)

In [731]:
captionString \
    = 'Table 2.5.1: Dummies DataFrame Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (dummiesDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

### **Split Preprocessed Data into Features and Target Arrays**

In [732]:
yNumpyArray \
    = dummiesDataFrame['IS_SUCCESSFUL'].values

log_function \
    .DebugReturnObjectWriteObject \
        (yNumpyArray)

In [733]:
XNumpyArray \
    = dummiesDataFrame \
        .drop \
            (['IS_SUCCESSFUL'],
             axis = 1) \
        .values

log_function \
    .DebugReturnObjectWriteObject \
        (XNumpyArray)

### **Split the Preprocessed Data into a Training and Testing Dataset**

In [734]:
XTrainNumpyArray, \
XTestNumpyArray, \
yTrainNumpyArray, \
yTestNumpyArray \
    = train_test_split \
            (XNumpyArray,
             yNumpyArray,
             random_state = 42)

In [735]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTrainNumpyArray)

In [736]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTestNumpyArray)

In [737]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTrainNumpyArray)

In [738]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTestNumpyArray)

### **Create a StandardScaler Instance**

In [739]:
currentStandardScalar \
    = StandardScaler()

log_function \
    .DebugReturnObjectWriteObject \
        (currentStandardScalar)

### **Fit the StandardScaler**

In [740]:
XStandardScalar \
    = currentStandardScalar \
        .fit \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XStandardScalar)

### **Scale the Data**

In [741]:
XTrainScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTrainScaledNumpyArray)

In [742]:
XTestScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTestNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTestScaledNumpyArray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [743]:
# These line of code define the model with the number of input features and hidden nodes for each layer.
neuralNetSequential \
    = tf.keras.models.Sequential()

log_function \
    .DebugReturnObjectWriteObject \
        (neuralNetSequential)

### **Layers**

In [744]:
inputDimensionInteger \
    = len(XTrainScaledNumpyArray[0])

# This line of code is the first layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 72,
              activation = 'sigmoid',
              input_dim = inputDimensionInteger))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 17,
              activation = 'sigmoid'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 15,
              activation = 'sigmoid'))

# This line of code is the output layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 1,
              activation = 'sigmoid'))

### **Check Model**

In [745]:
neuralNetSequential.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 72)                32616     
                                                                 
 dense_67 (Dense)            (None, 17)                1241      
                                                                 
 dense_68 (Dense)            (None, 15)                270       
                                                                 
 dense_69 (Dense)            (None, 1)                 16        
                                                                 
Total params: 34,143
Trainable params: 34,143
Non-trainable params: 0
_________________________________________________________________


### **Compile**

In [746]:
neuralNetSequential \
    .compile \
        (loss = 'binary_crossentropy',
         optimizer = 'adam',
         metrics = ['accuracy'])

## **3.2: Train Model**

In [747]:
neuralNetSequential \
    .fit \
        (XTrainScaledNumpyArray,
         yTrainNumpyArray,
         epochs = 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5257 - accuracy: 0.7517
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4416 - accuracy: 0.7912
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4309 - accuracy: 0.7942
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4266 - accuracy: 0.7946
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4240 - accuracy: 0.7941
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4225 - accuracy: 0.7948
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4206 - accuracy: 0.7959
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4197 - accuracy: 0.7961
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4188 - accuracy: 0.7956
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4186 - accura

## **3.3: Evaluate Model**

In [748]:
modelLossFloat, \
modelAccuracyFloat \
    = neuralNetSequential.evaluate \
        (XTestScaledNumpyArray,
         yTestNumpyArray,
         verbose = 2)

log_subroutine \
    .PrintAndLogWriteText \
        (f'\nModel Loss: {round(modelLossFloat * 100, 2)}%, '
         + f'Model Accuracy: {round(modelAccuracyFloat*100, 2)}%')

268/268 - 0s - loss: 0.4688 - accuracy: 0.7904 - 415ms/epoch - 2ms/step

Model Loss: 46.88%, Model Accuracy: 79.04%


## **3.4: Export Model**

In [749]:
neuralNetSequential.save \
    (CONSTANT_OPTIMIZATION_FILE_PATH)

In [750]:
#log_subroutine \
#    .EndProgramExecution()